앞의 실습에서 다룬 것처럼 LASSO Regression을 이용하여 키워드를 추출합니다.

In [1]:
import config
from navernews_10days import get_bow

x, _idx_to_vocab, _vocab_to_idx = get_bow(tokenize='noun', date='2016-10-20')

soynlp=0.0.49
added lovit_textmining_dataset


`_idx_to_vocab` 와 `_vocab_to_idx` 를 이용하여 단어를 인덱스로, 인덱스를 단어로 바꾸는 함수를 만들어 둡니다. 

In [2]:
def vocab_to_idx(word):
    return _vocab_to_idx.get(word, -1)

def idx_to_vocab(idx):
    if 0 <= idx < len(_idx_to_vocab):
        return _idx_to_vocab[idx]
    return None

vocab_to_idx('게이트')

1136

sparse matrix에서 nonzero() 함수는 0이 아닌 값을 가지는 (row, col)을 각각 row idx array, column idx array로 출력해 줍니다. 아래 예시에서는 (1, 200), (1, 219), (1,593), ... ,(30090, 7642), (30090, 9106), (30090, 3424)가 0이 아닌 값을 가진다는 의미입니다. 

In [3]:
x.nonzero()

(array([    1,     1,     1, ..., 30090, 30090, 30090], dtype=int32),
 array([ 200,  219,  593, ..., 7642, 9106, 3424], dtype=int32))

'아이오아이'라는 단어가 들어간 문서와 '아이오아이'가 들어가지 않은 문서를 구분하는 LASSO Regression을 학습하여, 두 집단을 구분하는데 유용한 단어를 keywords로 선택해 보겠습니다. 

x[:,word_idx]는 word_idx 컬럼만 선택하여 shape = (30090, 1) 모양의 submatrix를 만드는 작업입니다. 

여기에 x[:,word_idx].nonzero()[0]를 하면 word_idx, 즉 '아이오아이'라는 단어가 포함되어 있는 document row idx를 구할 수 있습니다. 이를 positive_document로 선택합니다. 

y는 positive documenet에 해당하면 1로, 그렇지 않으면 -1로 만듭니다. 즉, '아이오아이'라는 단어를 포함하면 positive class, 그렇지 않으면 negative class 입니다. 

In [4]:
idx = vocab_to_idx('아이오아이')
positive_document = x[:,idx].nonzero()[0]

In [5]:
import numpy as np
from collections import Counter

y = -np.ones(x.shape[0], dtype=np.int)
y[positive_document] = 1
print(Counter(y))

Counter({-1: 29994, 1: 97})


'아이오아이'라는 단어가 들어간 문서와 그렇지 않은 문서를 구분하는 가장 좋은 feature (단어)는 당연히 '아이오아이'입니다. 그렇다면 keyword를 추출할 때에는 sparse matrix, x에서 이 단어를 지워야 합니다. 

scipy.sparse.csr_matrix 는 다음과 같은 세 가지 array로 구성되어 있습니다. 

    row = [0, 0, 0, ...]   
    col = [263, 733, 9492, ... ]
    data = [3, 87, 3, , ...] 


data 는 (row, col)에 해당하는 term frequency 입니다. 이를 가져오는 부분은 아래와 같이 실행하면 됩니다. 

    (row, col) = x.nonzero()
    data = x.data
    
그리고 column에서 해당하는 단어, 즉 '아이오아이'에 해당하는 column 값이 있으면 이를 무시합니다. 

    for r, c, d in zip(row, col, data):
        if c == word_idx:
            continue
            
이제 우리는 '아이오아이'라는 단어의 column이 0인 새로운 sparse matrix를 만들었습니다. 

    x_train = csr_matrix((data_, (row_, col_)))
    
이 과정이 반복될테니 get_train_data(word) 이라는 함수로 만들어 둡니다. 

In [6]:
def get_train_data(word):
    idx = vocab_to_idx(word)
    positive_document = x[:,idx].nonzero()[0]

    y_train = -np.ones(x.shape[0], dtype=np.int)
    y_train[positive_document] = 1

    n_docs, n_terms = x.shape
    (row, col) = x.nonzero()
    data = x.data

    row_ = []
    col_ = []
    data_ = []

    for r, c, d in zip(row, col, data):
        if c == idx:
            continue
        row_.append(r)
        col_.append(c)
        data_.append(d)

    from scipy.sparse import csr_matrix
    x_train = csr_matrix((data_, (row_, col_)), shape=(n_docs, n_terms))
    
    return x_train, y_train

x_train, y_train = get_train_data('최순실')

x_train의 shape 자체는 바뀌지 않았습니다. 

In [7]:
x_train.shape

(30091, 9774)

앞선 예제처럼 penalty='L1'을 설정하여 LASSO Regression을 학습합니다. 

In [8]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=10)
logistic_l1.fit(x_train, y_train)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

coefficient가 높은 단어들을 상위 20개 출력해 봅니다. coefficient가 0에 가까우면 사실 큰 의미가 없는 단어이기 때문에, 어느 수준 이상의 coefficient 이상을 가지는 단어만을 선택합니다. 

    if coef < 0.001: break
    
2016-10-20 은 최순실 게이트가 세상에 드러나기 시작했던 날입니다. ['게이트', '고영태', 정유라'] 등의 단어가 키워드로 드러납니다. 

In [9]:
idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])

for idx, coef in sorted_coefficients[:20]:
    if coef < 0.001: break
    print(idx_to_vocab(idx))

게이트
고영태
최서원
조회
정유라
역전
모녀
일이다
침몰
편파기소
융성
원점
모색
씨와
꼬리
정면
지지부진
거의
지목
김세


이 과정 역시 반복되므로 lasso_keyword라는 함수로 묶어둡니다. word가 입력되면 x에서 이에 해당하는 column을 0으로 만들고, LASSO regression을 학습하여 coefficient가 높은 순으로 topk개를 단어로 선택합니다. 

In [10]:
def lasso_keyword(word, C=20, topk=50):
    if not (word in _vocab_to_idx):
        return []

    x_train, y_train = get_train_data(word)
    logistic_l1 = LogisticRegression(penalty='l1', C=C)
    logistic_l1.fit(x_train, y_train)

    idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
    sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])

    # filtering keyword
    keywords = [word_idx for word_idx, coef in
        sorted_coefficients[:topk] if coef > 0.001]

    # decode idx to str
    keywords = [idx_to_vocab(word_idx) for word_idx in keywords]
    return keywords

print(lasso_keyword('최순실'))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['게이트', '고영태', '최서원', '조회', '정유라', '역전', '원점', '일이다', '모녀', '침몰', '모색', '융성', '꼬리', '편파기소', '씨와', '정면', '화두', '거의', '지목', '김세', '정국', '지지부진', '가치', '불신', '제3지대', '문제없', '운명', '집권', '학사관리', '일고', '이화여대', '미르', '일을', '뉴스', '최동', '어렵게', '한겨레', '나라', '블루', '민생', '파악', '침묵', '노컷뉴스', '커지고', '정유라씨', '승마', '지도부', '연관성', '송민순', '미르재단']


C의 값을 바꿔가며 '아이오아이' 관련 문서들의 키워드를 추출합니다. 작은 C는 좀 더 강한 regularization을 한다는 의미이기 때문에 키워드의 개수가 줄어듭니다. 

In [11]:
print(lasso_keyword('아이오아이', C=20))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['너무너무너무', '산들', '선의', '엠카운트다운', '다비치', '손을', '선보', '열창', '뮤직', '사랑스', '만나게', '박진영', '챔피언', '세련', '먹고', '사나', '소속사', '같이', '파워풀', '당연', '곡으로', '보컬', '마무리', '뉴스1스타', '버전', '컴백', '걸크러쉬', '유연', '일산', '인사', '완전체', '1위', '순위', '소식', '서울신문', '부모님', '인기', '등장', '멤버들', '무대', '유정', '발매', '활동', '수준', '91', '수출', '드림', '헤럴드', '감사', '프로듀스101']


In [12]:
print(lasso_keyword('아이오아이', C=0.05))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


['너무너무너무', '엠카운트다운', '활동', '무대', '1위', '걸그룹', '불독', '다이아', '컴백', '방탄소년단', '타이틀곡', '프로듀스101', '순위', '신용재']


우리는 이 예제에서 하나의 단어를 입력하여 해당 단어 (예를 들어 '아이오아이')가 포함된 문서를 positive로 선택하였습니다만, 주어진 문서들의 id만 구할 수 있다면, 그 문서들과 다른 문서들을 구분하는 키워드를 선택할 수도 있습니다. 

이처럼 한 단어가 포함되어 있는 문서를 positive documents로 정의하면, 이 문서집합의 키워드는 해당 단어의 연관어 이기도 합니다. 


문서 군집화를 한 뒤, 해당 군집과 다른 군집을 구분하는 키워드를 선택하면 군집의 label이 되기도 합니다. 이 부분은 문서 군집화 수업 이후에 좀 더 이야기하겠습니다. 